In [47]:
# === Importación de librerías necesarias ===
import pandas as pd  
import numpy as np   
import matplotlib.pyplot as plt 
import seaborn as sns  
from sklearn.model_selection import train_test_split, GridSearchCV 
from sklearn.ensemble import RandomForestClassifier  
from sklearn.preprocessing import MinMaxScaler 
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


In [48]:
import pandas as pd

# Carga individual
df_primera = pd.read_csv('D:/TFG/codigo/datos/indicadores-filtrados-primera-5-cambios.csv', sep=",")
df_premier = pd.read_csv('D:/TFG/codigo/datos/indicadores-filtrados-premier-5-cambios.csv', sep=",")
df_bundesliga = pd.read_csv('D:/TFG/codigo/datos/indicadores-filtrados-bundesliga-5-cambios.csv', sep=",")


# Unir en un solo DataFrame
df = pd.concat([df_primera, df_premier, df_bundesliga], ignore_index=True)

# === Eliminación de columnas irrelevantes o identificadores ===
df.drop(['jornada', 'temporada', 'id_indicadores_equipo_prepartido', 'id_partido'], axis=1, inplace=True)
df.head()


,porcentaje local ganados en sitio,porcentaje local ganados en general,porcentaje local empatados en sitio,porcentaje local empatados en general,porcentaje local perdidos en sitio,porcentaje local perdidos en general,porcentaje visitante ganados en sitio,porcentaje visitante ganados en general,porcentaje visitante empatados en sitio,porcentaje visitante empatados en general,...,proporcion local cambios alineacion centrocampista en general,proporcion visitante cambios alineacion centrocampista en sitio,proporcion visitante cambios alineacion centrocampista en general,proporcion local cambios alineacion delantero sitio,proporcion local cambios alineacion delantero en general,proporcion visitante cambios alineacion delantero en sitio,proporcion visitante cambios alineacion delantero en general,resultado_local,resultado_visitante,resultado_partido
0,50.00,37.50,0.00,12.50,50.00,50.00,0.00,0.00,60.00,66.67,...,1.38,1.60,1.67,1.75,1.75,1.00,1.22,1,1,X
1,66.67,42.86,0.00,14.29,33.33,42.86,0.00,11.11,80.00,44.44,...,1.57,1.60,1.67,2.00,2.14,0.80,1.00,4,2,1
2,0.00,12.50,66.67,37.50,33.33,50.00,66.67,42.86,0.00,28.57,...,2.00,1.00,1.14,1.00,0.75,2.00,2.29,1,1,X
3,80.00,55.56,20.00,33.33,0.00,11.11,60.00,62.50,20.00,12.50,...,1.11,1.80,2.00,1.20,1.33,1.20,1.38,1,1,X
4,75.00,71.43,25.00,28.57,0.00,0.00,33.33,42.86,33.33,28.57,...,2.29,1.67,1.57,1.50,1.86,1.67,1.43,1,0,1


In [49]:
# === Separación de variables predictoras (X) y objetivos (y) ===

# X contendrá todos los indicadores que se usarán como predictores
X = df.drop(['resultado_partido', 'resultado_local', 'resultado_visitante'], axis=1)

# Variables objetivo posibles:
y_resultado = df['resultado_partido']            # Resultado general (1, 2 o X)
y_resultado_local = df['resultado_local']        # Resultado desde el punto de vista del equipo local (1 si gana el local, 0 en otro caso)
y_resultado_visitante = df['resultado_visitante']  # Idem para el visitante

# En este modelo trabajaremos con y_resultado

In [50]:
# === Escalado de variables predictoras ===
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)  # Escala todos los valores entre 0 y 1

### PREDICCIONES RESULTADO

In [51]:
# === División del conjunto de datos en entrenamiento y prueba ===
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_resultado, test_size=0.2, random_state=42
)


In [52]:
# === Definición del modelo base ===
model = RandomForestClassifier(random_state=42)

# === Definición del grid de hiperparámetros a probar ===
param_grid = {
    'n_estimators': [100, 150, 200],       # Número de árboles en el bosque
    'max_depth': [10, 20, 30],             # Profundidad máxima de los árboles
    'min_samples_split': [2, 5, 10]        # Mínimo de muestras para dividir un nodo interno
}

# === Búsqueda de los mejores hiperparámetros con validación cruzada (5 particiones) ===
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=5,                   # 5-fold cross-validation
    scoring='accuracy',     # Métrica de evaluación
    n_jobs=-1               # Usa todos los núcleos disponibles para paralelizar
)

# Ajustar el modelo con los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Extraer los mejores hiperparámetros
best_params = grid_search.best_params_
print("Mejores hiperparámetros encontrados:", best_params)


Mejores hiperparámetros encontrados: {'max_depth': 30, 'min_samples_split': 2, 'n_estimators': 100}


In [53]:
# === Entrenamiento del modelo con los mejores hiperparámetros ===
best_model = RandomForestClassifier(**best_params, random_state=42)
best_model.fit(X_train, y_train)

# === Predicción sobre el conjunto de prueba ===
y_test_pred = best_model.predict(X_test)


In [54]:
# === Evaluación de la precisión global del modelo ===
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Precisión (accuracy) en el conjunto de prueba: {test_accuracy:.4f}")

# === Matriz de confusión ===
conf_matrix = confusion_matrix(y_test, y_test_pred)
print("Matriz de confusión:")
print(conf_matrix)

Precisión (accuracy) en el conjunto de prueba: 0.4379
Matriz de confusión:
[[123  18  29]
 [ 64  15  19]
 [ 58  11  17]]


In [55]:
# === Cálculo de la importancia de cada variable ===
importances = best_model.feature_importances_
feature_names = X.columns

# Crear un DataFrame ordenado por importancia
feature_importance_df = pd.DataFrame({
    'Variable': feature_names,
    'Importancia': importances
}).sort_values(by='Importancia', ascending=False)

# Mostrar las 15 variables más importantes
print("Top 10 variables más importantes:")
print(feature_importance_df.head(10))


Top 10 variables más importantes:
                                             Variable  Importancia
82             proporcion visitante posesion en sitio     0.010117
81               proporcion local posesion en general     0.010038
83           proporcion visitante posesion en general     0.009995
13                 proporcion local puntos en general     0.009744
84              proporcion local total tiros en sitio     0.009099
85            proporcion local total tiros en general     0.008811
5                porcentaje local perdidos en general     0.008138
80                 proporcion local posesion en sitio     0.008137
47       porcentaje local mas 1,5 marcados en general     0.007961
95  proporcion visitante corners en contra en general     0.007844


In [56]:
# === Suma de la importancia de variables relacionadas con la rotación ===
cambio_vars = feature_importance_df[feature_importance_df['Variable'].str.contains('cambio', case=False)]
suma_importancia_cambio = cambio_vars['Importancia'].sum()
print(f"Suma total de importancia de las variables relacionadas con la rotación: {suma_importancia_cambio:.4f}")

Suma total de importancia de las variables relacionadas con la rotación: 0.4348


### GOLES DEL EQUIPO LOCAL

In [57]:
# === División del conjunto de datos en entrenamiento y prueba ===
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_resultado_local, test_size=0.2, random_state=42
)

In [58]:
# === Definición del modelo base ===
model = RandomForestClassifier(random_state=42)

# === Definición del grid de hiperparámetros a probar ===
param_grid = {
    'n_estimators': [100, 150, 200],       # Número de árboles en el bosque
    'max_depth': [10, 20, 30],             # Profundidad máxima de los árboles
    'min_samples_split': [2, 5, 10]        # Mínimo de muestras para dividir un nodo interno
}

# === Búsqueda de los mejores hiperparámetros con validación cruzada (5 particiones) ===
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=5,                   # 5-fold cross-validation
    scoring='accuracy',     # Métrica de evaluación
    n_jobs=-1               # Usa todos los núcleos disponibles para paralelizar
)

# Ajustar el modelo con los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Extraer los mejores hiperparámetros
best_params = grid_search.best_params_
print("Mejores hiperparámetros encontrados:", best_params)

c:\Users\Miguel\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


Mejores hiperparámetros encontrados: {'max_depth': 20, 'min_samples_split': 10, 'n_estimators': 150}


In [59]:
# === Entrenamiento del modelo con los mejores hiperparámetros ===
best_model = RandomForestClassifier(**best_params, random_state=42)
best_model.fit(X_train, y_train)

# === Predicción sobre el conjunto de prueba ===
y_test_pred = best_model.predict(X_test)

In [60]:
# === Evaluación de la precisión global del modelo ===
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Precisión (accuracy) en el conjunto de prueba: {test_accuracy:.4f}")

# === Matriz de confusión ===
conf_matrix = confusion_matrix(y_test, y_test_pred)
print("Matriz de confusión:")
print(conf_matrix)

Precisión (accuracy) en el conjunto de prueba: 0.2853
Matriz de confusión:
[[ 5 49 17  1  0  0  0  0]
 [19 71 33  1  0  0  0  0]
 [ 4 51 24  2  0  0  0  0]
 [ 3 32 15  1  0  0  0  0]
 [ 2  9  4  2  0  0  0  0]
 [ 0  5  2  0  0  0  0  0]
 [ 0  1  0  0  0  0  0  0]
 [ 0  1  0  0  0  0  0  0]]


In [61]:
# === Cálculo de la importancia de cada variable ===
importances = best_model.feature_importances_
feature_names = X.columns

# Crear un DataFrame ordenado por importancia
feature_importance_df = pd.DataFrame({
    'Variable': feature_names,
    'Importancia': importances
}).sort_values(by='Importancia', ascending=False)

# Mostrar las 15 variables más importantes
print("Top 10 variables más importantes:")
print(feature_importance_df.head(10))

Top 10 variables más importantes:
                                              Variable  Importancia
94     proporcion visitante corners en contra en sitio     0.008967
80                  proporcion local posesion en sitio     0.008736
85             proporcion local total tiros en general     0.008720
95   proporcion visitante corners en contra en general     0.008396
34            proporcion local goles marcados en sitio     0.008354
161  proporcion local cambios alineacion centrocamp...     0.008198
87         proporcion visitante total tiros en general     0.007968
167  proporcion visitante cambios alineacion delant...     0.007771
119  proporcion visitante cambios delanteros a cent...     0.007718
81                proporcion local posesion en general     0.007666


In [62]:
# === Suma de la importancia de variables relacionadas con la rotación ===
cambio_vars = feature_importance_df[feature_importance_df['Variable'].str.contains('cambio', case=False)]
suma_importancia_cambio = cambio_vars['Importancia'].sum()
print(f"Suma total de importancia de las variables relacionadas con la rotación: {suma_importancia_cambio:.4f}")

Suma total de importancia de las variables relacionadas con la rotación: 0.4504


### GOLES DEL EQUIPO VISITANTE

In [63]:
# === División del conjunto de datos en entrenamiento y prueba ===
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_resultado_visitante, test_size=0.2, random_state=42
)

In [64]:
# === Definición del modelo base ===
model = RandomForestClassifier(random_state=42)

# === Definición del grid de hiperparámetros a probar ===
param_grid = {
    'n_estimators': [100, 150, 200],       # Número de árboles en el bosque
    'max_depth': [10, 20, 30],             # Profundidad máxima de los árboles
    'min_samples_split': [2, 5, 10]        # Mínimo de muestras para dividir un nodo interno
}

# === Búsqueda de los mejores hiperparámetros con validación cruzada (5 particiones) ===
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=5,                   # 5-fold cross-validation
    scoring='accuracy',     # Métrica de evaluación
    n_jobs=-1               # Usa todos los núcleos disponibles para paralelizar
)

# Ajustar el modelo con los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Extraer los mejores hiperparámetros
best_params = grid_search.best_params_
print("Mejores hiperparámetros encontrados:", best_params)

c:\Users\Miguel\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


Mejores hiperparámetros encontrados: {'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 200}


In [65]:
# === Entrenamiento del modelo con los mejores hiperparámetros ===
best_model = RandomForestClassifier(**best_params, random_state=42)
best_model.fit(X_train, y_train)

# === Predicción sobre el conjunto de prueba ===
y_test_pred = best_model.predict(X_test)

In [66]:
# === Evaluación de la precisión global del modelo ===
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Precisión (accuracy) en el conjunto de prueba: {test_accuracy:.4f}")

# === Matriz de confusión ===
conf_matrix = confusion_matrix(y_test, y_test_pred)
print("Matriz de confusión:")
print(conf_matrix)

Precisión (accuracy) en el conjunto de prueba: 0.3418
Matriz de confusión:
[[37 77  2  0  0  0]
 [35 81  4  1  0  0]
 [16 61  3  0  0  0]
 [ 4 22  2  0  0  0]
 [ 1  5  0  0  0  0]
 [ 1  2  0  0  0  0]]


In [67]:
# === Cálculo de la importancia de cada variable ===
importances = best_model.feature_importances_
feature_names = X.columns

# Crear un DataFrame ordenado por importancia
feature_importance_df = pd.DataFrame({
    'Variable': feature_names,
    'Importancia': importances
}).sort_values(by='Importancia', ascending=False)

# Mostrar las 15 variables más importantes
print("Top 10 variables más importantes:")
print(feature_importance_df.head(10))

Top 10 variables más importantes:
                                              Variable  Importancia
86           proporcion visitante total tiros en sitio     0.007925
89         proporcion local corners a favor en general     0.007583
87         proporcion visitante total tiros en general     0.007571
156  proporcion local cambios alineacion defensa sitio     0.007460
159  proporcion visitante cambios alineacion defens...     0.007405
115         media visitante cambios minutos en general     0.007268
112                  media local cambios minutos sitio     0.007246
82              proporcion visitante posesion en sitio     0.007235
69               proporcion local amarillas en general     0.007225
149        proporcion local cambios 61 a 75 en general     0.007192


In [68]:
# === Suma de la importancia de variables relacionadas con la rotación ===
cambio_vars = feature_importance_df[feature_importance_df['Variable'].str.contains('cambio', case=False)]
suma_importancia_cambio = cambio_vars['Importancia'].sum()
print(f"Suma total de importancia de las variables relacionadas con la rotación: {suma_importancia_cambio:.4f}")

Suma total de importancia de las variables relacionadas con la rotación: 0.4588
